Декоратор - это вызываемый объект, который принимает другую функцию в качестве аргумента (декорируемую функцию). Декоратор может производить какие-то операции с функцией и возвращает либо её саму, либо другую заменяющую её функцию или вызываемый объект

* Декоратор - это функция или другой вызываемый объект
* Декоратор может заменить декорируемую функцию другой
* Декораторы выполняются сразу после загрузки модуля (сами декорируемые функции - только в результате явного вызова)

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner


In [2]:
# Код:
@deco
def target():
    print('running target()')

# Эквивалентен коду
def target():
    print('running target()')

target = deco(target)

In [3]:
target()

running inner()


Правила видимости переменных

In [4]:
b = 6
def f2(a):
    print(a)
    print(b) # Всякая переменная, которая присваивается в теле функции, является локальной
    b = 9
f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

Сравнение байт-кода

In [ ]:
def f1(a):
    print(a)
    print(b)

b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [ ]:
from dis import dis

In [ ]:
dis(f1)

  1           0 RESUME                   0

  2           2 LOAD_GLOBAL              1 (NULL + print)
             14 LOAD_FAST                0 (a)
             16 PRECALL                  1
             20 CALL                     1
             30 POP_TOP

  3          32 LOAD_GLOBAL              1 (NULL + print)
             44 LOAD_GLOBAL              2 (b)
             56 PRECALL                  1
             60 CALL                     1
             70 POP_TOP
             72 LOAD_CONST               0 (None)
             74 RETURN_VALUE


In [ ]:
dis(f2)

  6           0 RESUME                   0

  7           2 LOAD_GLOBAL              1 (NULL + print)
             14 LOAD_FAST                0 (a)
             16 PRECALL                  1
             20 CALL                     1
             30 POP_TOP

  8          32 LOAD_GLOBAL              1 (NULL + print)
             44 LOAD_FAST                1 (b)
             46 PRECALL                  1
             50 CALL                     1
             60 POP_TOP

  9          62 LOAD_CONST               1 (9)
             64 STORE_FAST               1 (b)
             66 LOAD_CONST               0 (None)
             68 RETURN_VALUE


Замыкания
* Замыкание - это функция, назовем ее f, с расширенной областью видимости, которая охватывает переменные, на которые есть ссылки в теле f, но которые не являются ни глобальными, ни локальными переменными f. Такие переменные должны происходить из локальной области видимости внешней функции, объемлющей f.

In [ ]:
# Пример замыкания
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

In [ ]:
avg = make_averager()
avg(1)
avg(2)
avg2 = make_averager()
avg2(10)


10.0

In [ ]:
def make_averager_v2():
    series = []
    def avarager(new_value):
        series += [2]
        total = sum(series)
        return total/ len(series)
    return avarager

In [ ]:
avg3 = make_averager_v2()
avg3(1)

UnboundLocalError: cannot access local variable 'series' where it is not associated with a value

Логика поиска переменных
* Если имеется объявление global x, то x берётся из него и присваивается глобальной переменной x уровня модуля;
* Если имеется объявление nonlocal x, то x берётся из него и присваивается локальной переменной x в ближайшей объемлющей функции, в которой x определена;
* Если x - параметр или её присвоено значение в теле функции, то x - локальная переменная;


In [ ]:
def test():
    global i_am_from_test_global
    i_am_from_test = 2

test()
print(i_am_from_test_global)

SyntaxError: no binding for nonlocal 'i_am_from_test_nonlocal' found (668899564.py, line 3)

Реализация простого декоратора

In [ ]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = [repr(arg) for arg in args]
        arg_lst.extend(f'{k}={v!r}' for k, v in kwargs.items())
        arg_str = ", ".join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

In [ ]:
@clock
def some_test():
    test = range(1, 10000000)
    return sum(test)

In [ ]:
some_test()
print(some_test.__name__)

[0.49324320s] some_test() -> 49999995000000
some_test


# Декораторы в стандартной библиотеке


* functools.cache - реализует запоминание (memoization): сохранение результатов предыдущих вызовов для избежания повторного вычисления. Работает с версии 3.9. Обёртка для functools.lru_cache;
* functools.lru_cache - делает то же самое, что и functools.cache, но можно ограничить число сохраняемых результатов (по умолчанию 128). Элементы, к которым давно не было обращений, вытесняются, чтобы освободить место для новых. lru - least recently used

In [ ]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [ ]:
fibonacci(6)

[0.00000090s] fibonacci(0) -> 0
[0.00000140s] fibonacci(1) -> 1
[0.00015290s] fibonacci(2) -> 1
[0.00000060s] fibonacci(1) -> 1
[0.00000070s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00003450s] fibonacci(2) -> 1
[0.00007020s] fibonacci(3) -> 2
[0.00025400s] fibonacci(4) -> 3
[0.00000040s] fibonacci(1) -> 1
[0.00000030s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00002460s] fibonacci(2) -> 1
[0.00004940s] fibonacci(3) -> 2
[0.00000040s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00002570s] fibonacci(2) -> 1
[0.00000030s] fibonacci(1) -> 1
[0.00000050s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00002620s] fibonacci(2) -> 1
[0.00005200s] fibonacci(3) -> 2
[0.00010300s] fibonacci(4) -> 3
[0.00017770s] fibonacci(5) -> 5
[0.00045860s] fibonacci(6) -> 8


8

In [ ]:
@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [ ]:
fibonacci(6)

[0.00000050s] fibonacci(0) -> 0
[0.00000150s] fibonacci(1) -> 1
[0.00039150s] fibonacci(2) -> 1
[0.00000110s] fibonacci(3) -> 2
[0.00042300s] fibonacci(4) -> 3
[0.00000100s] fibonacci(5) -> 5
[0.00045270s] fibonacci(6) -> 8


8

* functools.singledispatch - декоратор, который можно использовать для имитации перегрузки функции. Обычная функция, декорированная @singledispatch, становится точкой входа для обобщённой функции: группы функций, выполняющих одну и ту же логическую операцию по-разному в зависимости от типа первого аргумента. Это называется одиночной диспетчеризацией. 

In [ ]:
from functools import singledispatch

class GameCharacter:
    def __init__(self) -> None:
        self.status = "alive"
    def __repr__(self) -> str:
        return f'GameCharacter()'
        
class Thief(GameCharacter):
    def __init__(self) -> None:
        super().__init__()
    def __repr__(self) -> str:
        return f'Thief()'

class Wizard(GameCharacter):
    def __init__(self) -> None:
        super().__init__()
    def __repr__(self) -> str:
        return f'Wizard()'

@singledispatch
def trap(game_character: GameCharacter):
    game_character.status = 'disappeared'

@trap.register
def _(game_character: Thief):
    # Ничего не делаем, так как вору ловушки нипочём
    pass

@trap.register
def _(game_character: Wizard):
    # Для волшебника не так всё радужно
    game_character.status = 'dead'



In [ ]:
game_characters = [Wizard(), Thief(), GameCharacter()]
for game_character in game_characters:
    trap(game_character)
    print(game_character, game_character.status)

Wizard() dead
Thief() alive
GameCharacter() disappeared


In [ ]:
from functools import singledispatch

@singledispatch
def method(a) -> str:
    return "default"

@method.register
def _(b) -> str:
    return "dispatch"

class A:
    pass

@method.register
def _(aObject: A) -> str:
    return "A"

print(method(A()))
print(method("bla bla"))
print(method(object()))

A
dispatch
default


Параметризованные декораторы

In [ ]:
# Без параметров

registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x000001f8bea0eb60>)
running main()
registry -> [<function f1 at 0x000001f8bea0eb60>]
running f1()


In [5]:
import registration_param

running register (active=False)->decorate(<function f1 at 0x0000029c28dce700>)
running register (active=True)->decorate(<function f2 at 0x0000029c28f10cc0>)


In [7]:
def f3():
    print('running f3()')

In [14]:
print(registration_param.registry)
registration_param.register()(f3)
print(registration_param.registry)
registration_param.register(active=False)(f3)
print(registration_param.registry)

{<function f3 at 0x0000029c29a14220>, <function f2 at 0x0000029c28f10cc0>}
running register (active=True)->decorate(<function f3 at 0x0000029c29a14220>)
{<function f3 at 0x0000029c29a14220>, <function f2 at 0x0000029c28f10cc0>}
running register (active=False)->decorate(<function f3 at 0x0000029c29a14220>)
{<function f2 at 0x0000029c28f10cc0>}


In [16]:
import time
from clockdeco_param import clock

@clock('time = {elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

time = 0.124s
time = 0.124s
time = 0.126s
